In [ ]:
import re
import os
import os.path as op
from pprint import pprint
from osl import utils
from osl import source_recon
import numpy as np
import coinsmeg_data as coinsmeg

The first thing we need to do is to use the structural image to compute the head and brain surfaces in Native and MNI space using *rhino.compute_surfaces*. The head and brain surfaces will be used later  for carrying out the coregistration and forward modelling.

For now, we'll do this for one subject.

The inputs we need to provide are for the first subject are:

* *smri_file* - the full path to the structural MRI niftii file
* *recon_dir* - the full path to the directory that will contain the subject directories that RHINO will output
* *subject* - the name of the subject directory that RHINO will output
* *include_nose* - a boolean flag indicating whether or not to extract a head surface from the structural MRI that includes the nose. It your structural MRI includes the nose AND you have acquired polhemus headshape points that include the nose, then it is recommend to set this flag to True

In [10]:
sub = coinsmeg.sub_num2str(17) #  function sub_num2str converts a numerical subject identifier into a string formatted as sub-XX

# Directories

preproc_dir = op.join(coinsmeg.DERIVATIVES_DIR, "preprocessed")
recon_dir = op.join(coinsmeg.DERIVATIVES_DIR, "recon")

print(preproc_dir)
anat_dir = coinsmeg.get_sub_anat_dir(sub)
smri_file = f"{anat_dir}/{sub}_T1w.nii"

# There are multiple runs for each subject. We will first fetch all data using an OSL utility
def find_fif_files(data_dir, subject, run=None):
    # Define the base search pattern
    base_pattern = f"{subject}_ses-2-meg_task-coinsmeg_run-"
    
    # If run is specified, use the exact pattern
    if run:
        search_pattern = f"{base_pattern}{run}_meg_transsss_preproc_raw"
    else:
        # If run is not specified, use a wildcard pattern
        search_pattern = f"{base_pattern}.*_meg_transsss_preproc_raw"

    print(search_pattern)
    # List to store paths of matching files
    matching_files = []

    # Walk through the directory recursively
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith(".fif"):
                # Match using regex to account for wildcard if run is not specified
                if re.search(search_pattern, file):
                    matching_files.append(os.path.join(root, file))
    
    return matching_files

fif_files = find_fif_files(preproc_dir, sub)


print('Structural files:')
pprint(smri_file)

print('fif files:')
pprint(fif_files)

/ohba/pi/lhunt/datasets/coins-meg_data/derivatives/preprocessed
sub-17_ses-2-meg_task-coinsmeg_run-.*_meg_transsss_preproc_raw
Structural files:
'/ohba/pi/lhunt/datasets/coins-meg_data/sub-17/ses-3-structural/anat/sub-17_T1w.nii'
fif files:
['/ohba/pi/lhunt/datasets/coins-meg_data/derivatives/preprocessed/sub-17_ses-2-meg_task-coinsmeg_run-4_meg_transsss/sub-17_ses-2-meg_task-coinsmeg_run-4_meg_transsss_preproc_raw.fif',
 '/ohba/pi/lhunt/datasets/coins-meg_data/derivatives/preprocessed/sub-17_ses-2-meg_task-coinsmeg_run-1_meg_transsss/sub-17_ses-2-meg_task-coinsmeg_run-1_meg_transsss_preproc_raw.fif',
 '/ohba/pi/lhunt/datasets/coins-meg_data/derivatives/preprocessed/sub-17_ses-2-meg_task-coinsmeg_run-2_meg_transsss/sub-17_ses-2-meg_task-coinsmeg_run-2_meg_transsss_preproc_raw.fif',
 '/ohba/pi/lhunt/datasets/coins-meg_data/derivatives/preprocessed/sub-17_ses-2-meg_task-coinsmeg_run-3_meg_transsss/sub-17_ses-2-meg_task-coinsmeg_run-3_meg_transsss_preproc_raw.fif']


In [11]:
source_recon.rhino.compute_surfaces(
    smri_file,
    recon_dir,
    sub,
    include_nose=True,
)

*** RUNNING OSL RHINO COMPUTE SURFACES ***
The nose is going to be added to the outer skin (scalp) surface.
Please ensure that the structural MRI has a FOV that includes the nose
reorienting subject brain to be RADIOLOGICAL
You can use the following call to check the passed in structural MRI is appropriate,
including checking that the L-R, S-I, A-P labels are sensible:
In Python:
fsleyes("/ohba/pi/lhunt/datasets/coins-meg_data/derivatives/recon/sub-17/rhino/surfaces/smri.nii.gz", "/home/ali/fsl/data/standard/MNI152_T1_1mm_brain.nii.gz")
From the cmd line:
fsleyes /ohba/pi/lhunt/datasets/coins-meg_data/derivatives/recon/sub-17/rhino/surfaces/smri.nii.gz /home/ali/fsl/data/standard/MNI152_T1_1mm_brain.nii.gz
Running BET pre-FLIRT...
Running FLIRT...
Running BET and BETSURF...
Refining scalp surface...
Adding nose to scalp surface...
rhino.surfaces.surfaces_display("/ohba/pi/lhunt/datasets/coins-meg_data/derivatives/recon", "sub-17") can be used to check the result
*** OSL RHINO COMPUTE S

False

We can now view the result using *fsleyes*. Note that *fsleyes* can sometimes take a few moments to open.

CHECK: in fsleyes that:

* The surfaces have been extracted properly compared with the structural
* The nose is included in the scalp surface, if that was requested with the *include_nose* option above

In [12]:
source_recon.rhino.surfaces_display(recon_dir, sub)

We need to now provide the coordinates for the nasion, LPA, RPA, and headshape points in Polhemus (head) space, in millimetres. The polhemus coordinates need to be extracted from the MEG fif file.